In [ ]:
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import requests
    import json
except ImportError:
    %pip install pandas numpy matplotlib requests json
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import requests
    import json


In [ ]:
import subprocess
cmd_status = ["docker", "compose", "ps", "-q"]
if not subprocess.run(cmd_status, capture_output=True, text=True).stdout.strip():
    subprocess.run(["docker", "compose", "up", "-d"], check=True)
    print("Docker Compose started.")
else:
    print("Docker Compose already running.")


# ==========================================
# 1. THE "PLANT" (Simulation Environment)
# ==========================================

In [ ]:
class SmartHomeEnv:
    def __init__(self, battery_capacity=10.0, max_power=3, 
                 heat_storage_capacity=20.0, hp_max_power=3.0, cop=3.0,
                 seed=42, hours=72):
        self.battery_capacity = battery_capacity # kWh
        self.max_power = max_power # kW
        self.soc = 5 # Initial State of Charge (kWh)
        self.initial_soc = self.soc
        
        # Heat System
        self.heat_storage_capacity = heat_storage_capacity # kWh (thermal)
        self.hp_max_power = hp_max_power # kW (electric)
        self.cop = cop
        self.heat_soc = 5.0 # Initial Heat SoC
        self.initial_heat_soc = self.heat_soc
        
        self.time_step = 0

        # Set a fixed seed for reproducibility
        np.random.seed(seed)

        # Internal Data Generation (The "Real World")
        # We now use the minute-level data as the primary simulation data
        self.data_hourly, self.data = self._generate_scenario_data(hours=hours)

    def _generate_scenario_data(self, hours):
        minutes = hours * 60
        t_min = np.arange(minutes)
        t_hours = t_min / 60.0

        # Solar: Peak at noon + random clouds
        solar = np.maximum(0, 5 * np.sin(2 * np.pi * (t_hours - 6) / 24))
        solar = np.maximum(0, solar - 0.3 * np.random.weibull(0.5, size=minutes))

        # Load: Morning/Evening peaks
        load = 2 + np.cos(4 * np.pi * (t_hours - 18) / 24) + \
                0.8 * np.cos(2 * np.pi * (t_hours - 14) / 24)
        load = np.maximum(0.5, load)
        # Price: High in evening
        price = 0.20 + 0.3 * np.cos(2 * np.pi * (t_hours - 18) / 24)
        
        # Sell Price: Fixed feed-in tariff (e.g. 0.10 EUR/kWh)
        sell_price = np.minimum(price,0.10 * np.ones(minutes)) - 0.1
        # sell_price = price - 0.1

        # --- Heat Demand Generation ---
        # Baseload
        heat_demand = np.ones(minutes) * 0.5 
        # Morning Block (e.g., 6-9)
        morning_mask = ((t_hours % 24) >= 6) & ((t_hours % 24) < 9)
        heat_demand[morning_mask] += 2.0
        # Evening Block (e.g., 18-22)
        evening_mask = ((t_hours % 24) >= 18) & ((t_hours % 24) < 22)
        heat_demand[evening_mask] += 3.0
        
        df_min = pd.DataFrame({
            'solar': solar, 
            'load': load, 
            'price': price, 
            'sell_price': sell_price,
            'heat_demand': heat_demand
        })
        
        # Aggregate to hourly
        # We group by integer division of index by 60 to get hourly blocks
        df_hourly = df_min.groupby(df_min.index // 60).mean()

        return df_hourly, df_min

    def reset(self):
        self.soc = self.initial_soc
        self.heat_soc = self.initial_heat_soc
        self.time_step = 0
        return self.data.iloc[0]

    def step(self, action):
        """
        Executes one time step.
        Args:
            action: 
                float: Desired battery power (legacy)
                dict: {'battery': float, 'heat_pump': float, 'pv_limit': float}
        """
        # Parse Action
        if isinstance(action, dict):
            bat_action_kw = action.get('battery', 0.0)
            hp_action_kw = action.get('heat_pump', 0.0)
            pv_limit_kw = action.get('pv_limit', float('inf'))
        else:
            bat_action_kw = float(action)
            hp_action_kw = 0.0
            pv_limit_kw = float('inf')

        current_data = self.data.iloc[self.time_step]

        # --- PV Curtailment ---
        available_solar = current_data['solar']
        actual_solar = min(available_solar, pv_limit_kw)
        curtailed_solar = available_solar - actual_solar

        # --- 1. Battery Physics ---
        # A. Power Limits
        power = np.clip(bat_action_kw, -self.max_power, self.max_power)

        # B. Capacity Limits
        # Energy change is Power * Time. Time is 1 minute (1/60 hour).
        energy_change = power / 60.0
        
        if energy_change > 0: # Charging
            max_charge = self.battery_capacity - self.soc
            if energy_change > max_charge:
                energy_change = max_charge
                power = energy_change * 60.0
        else: # Discharging
            max_discharge = self.soc
            if -energy_change > max_discharge:
                energy_change = -max_discharge
                power = energy_change * 60.0

        self.soc += energy_change

        # --- 2. Heat Pump Physics ---
        # HP Power Limit (Electric side)
        hp_power_elec = np.clip(hp_action_kw, 0, self.hp_max_power)
        
        # Heat Production
        heat_produced = hp_power_elec * self.cop
        heat_demand = current_data['heat_demand']
        
        # Heat Storage Logic
        # Energy balance: Storage_new = Storage_old + (Production - Demand) * dt
        heat_energy_change = (heat_produced - heat_demand) / 60.0
        
        # Apply Capacity Limits
        if heat_energy_change > 0: # Adding heat
            max_heat_add = self.heat_storage_capacity - self.heat_soc
            if heat_energy_change > max_heat_add:
                # Limit HP power to fit storage
                # heat_energy_change = (hp*cop - demand)/60 = max_heat_add
                # hp*cop = max_heat_add*60 + demand
                max_hp_elec = (max_heat_add * 60.0 + heat_demand) / self.cop
                hp_power_elec = min(hp_power_elec, max_hp_elec)
                
                # Recalculate actual change
                heat_produced = hp_power_elec * self.cop
                heat_energy_change = (heat_produced - heat_demand) / 60.0
                
        else: # Removing heat
            max_heat_remove = self.heat_soc
            if -heat_energy_change > max_heat_remove:
                # Storage empty! Cannot meet demand.
                heat_energy_change = -max_heat_remove

        self.heat_soc += heat_energy_change

        # --- 3. Calculate Cost ---
        # Grid Balance: Load + Charge + HP = Solar + Discharge + Grid
        # Grid = (Load - Solar) + Bat_Power + HP_Power
        net_load = current_data['load'] - actual_solar
        # Note: 'power' is battery power (+Charge, -Discharge)
        # If power > 0 (Charge), it adds to load. If power < 0 (Discharge), it subtracts.
        grid_kw = net_load + power + hp_power_elec

        # Cost is for the energy consumed in this minute
        grid_kwh = grid_kw / 60.0
        
        if grid_kw > 0:
            cost = grid_kwh * current_data['price']
        else:
            cost = grid_kwh * current_data['sell_price']

        # --- 4. Prepare Next Step ---
        self.time_step += 1
        done = self.time_step >= len(self.data)

        next_obs = None
        if not done:
            next_obs = self.data.iloc[self.time_step]

        info = {
            'soc': self.soc,
            'grid_kw': grid_kw,
            'battery_action_actual': power,
            'hp_power': hp_power_elec,
            'heat_soc': self.heat_soc,
            'heat_demand': heat_demand,
            'load': current_data['load'],
            'solar': actual_solar,
            'solar_potential': available_solar,
            'curtailed_solar': curtailed_solar,
            'price': current_data['price'],
            'sell_price': current_data['sell_price']
        }

        return next_obs, cost, done, info

    def get_forecast(self, horizon=24):
        """Returns the data for the next N steps (minutes)"""
        start = self.time_step
        end = min(start + horizon, len(self.data))
        return self.data.iloc[start:end]

    def get_hourly_forecast(self, horizon=24):
        """Returns the hourly data for the next N hours"""
        current_hour = self.time_step // 60
        start = current_hour
        end = min(start + horizon, len(self.data_hourly))
        return self.data_hourly.iloc[start:end]

# ==========================================
# 2. THE OPTIMIZERS (Hourly)
# ==========================================

In [ ]:
class Optimizer:
    """Base optimizer: stores battery capacity and max power values and defines interface."""
    def __init__(self, env):
        self.bat_cap = env.battery_capacity
        self.max_p = env.max_power
        self.env = env

    def get_setpoint(self, observation, current_soc, current_heat_soc=None):
        """
        Calculate the setpoint for the next hour.
        Args:
            observation: Hourly aggregated data for the current hour.
            current_soc: Current Battery State of Charge.
            current_heat_soc: Current Heat State of Charge.
        Returns:
            dict: {'battery': kW, 'heat_pump': kW}
        """
        raise NotImplementedError

In [ ]:
class ResidualChargeOptimizer(Optimizer):
    """Store excess solar; discharge to meet deficits (Hourly Average)."""

    def get_setpoint(self, observation, current_soc, current_heat_soc=None):
        net_load = observation['load'] - observation['solar']
        if net_load < 0:
            bat_action = min(self.max_p, -net_load)
        else:
            bat_action = -min(self.max_p, net_load)
            
        return {'battery': bat_action, 'heat_pump': 0.0}

In [ ]:
class RuleBasedOptimizer(Optimizer):
    """
    Rule-based optimizer:
    1. Tank Empty? -> Run HP (Grid if needed)
    2. Free Solar? -> Fill Tank
    3. Tank Full & Sun? -> Charge Battery
    4. Battery Full & Sun? -> Curtail PV
    """
    def get_setpoint(self, observation, current_soc, current_heat_soc=None):
        if current_heat_soc is None: current_heat_soc = 0.0
        
        # Initialize actions
        hp_action = 0.0
        bat_action = 0.0
        pv_limit = float('inf')
        
        # --- Rule 1: Is Tank Empty? ---
        # We define "Empty" as below a critical threshold, e.g., 10% or just enough to cover demand.
        # Let's say if heat_soc < 2.0 kWh (10% of 20kWh), we force HP to run.
        min_heat_soc = 2.0
        if current_heat_soc < min_heat_soc:
            # Run HP at max power to recover
            hp_action = self.env.hp_max_power
            
        # Calculate Excess Solar after mandatory HP run
        # Note: observation is hourly average.
        # If we forced HP, it consumes power.
        # Net Load = Load + HP - Solar
        # Excess Solar = Solar - Load - HP
        
        solar = observation['solar']
        load = observation['load']
        
        # Available solar for "Free Solar" logic
        # If hp_action was forced, it consumes some solar (or grid).
        # If hp_action was NOT forced (0.0), we have full solar - load.
        
        excess_solar = solar - load - hp_action
        
        if excess_solar > 0:
            # We have free solar!
            
            # --- Rule 2: Fill Tank ---
            # If we haven't already maxed out HP in Rule 1
            if hp_action < self.env.hp_max_power:
                # How much more can we put in?
                heat_room = self.env.heat_storage_capacity - current_heat_soc
                # Convert to electric power needed
                max_hp_elec_fill = heat_room / self.env.cop
                
                # We can use up to excess_solar, up to max_hp_power, up to fill tank
                hp_add = min(excess_solar, self.env.hp_max_power - hp_action, max_hp_elec_fill)
                
                hp_action += hp_add
                excess_solar -= hp_add
                
            # --- Rule 3: Charge Battery ---
            if excess_solar > 0:
                # How much room in battery?
                bat_room = self.bat_cap - current_soc
                
                # Charge with remaining excess
                bat_charge = min(excess_solar, self.max_p, bat_room)
                
                bat_action = bat_charge
                excess_solar -= bat_charge
                
                # --- Rule 4: Curtail PV ---
                if excess_solar > 0:
                    # We still have excess solar. Limit PV generation.
                    # Allowed Generation = Load + HP + Battery
                    pv_limit = load + hp_action + bat_action
                    
        else:
            # No excess solar (Deficit)
            # We might have a deficit because of Load or because of Rule 1 (Forced HP)
            deficit = -excess_solar # (Load + HP - Solar)
            
            # Discharge battery to meet deficit if possible
            if deficit > 0:
                bat_discharge = min(deficit, self.max_p, current_soc)
                bat_action = -bat_discharge
                
        return {'battery': bat_action, 'heat_pump': hp_action, 'pv_limit': pv_limit}

In [ ]:
class UrbsMPCOptimizer(Optimizer):
    """MPC optimizer using Urbs for optimization with perfect foresight."""
    def __init__(self, env, url="http://localhost:5000/simulate"):
        super().__init__(env)
        self.url = url
        self.plan = None
        self.horizon = 10000
        self.plan_initial_soc = None
        self.plan_initial_heat_soc = None

    def get_setpoint(self, observation, current_soc, current_heat_soc=None):
        if current_heat_soc is None: current_heat_soc = 0.0

        # Plan once at the beginning (Perfect Foresight)
        if self.plan is None:
            self.plan = self._run_optimization(horizon=self.horizon, initial_soc=current_soc, initial_heat_soc=current_heat_soc)
            self.plan_initial_soc = current_soc
            self.plan_initial_heat_soc = current_heat_soc
        
        # Get action for current timestep (Hour)
        t_hour = self.env.time_step // 60
        
        if t_hour < len(self.plan['battery']):
            # Closed-Loop execution (SoC Tracking) for Battery
            target_soc_end = self.plan_initial_soc + np.sum(self.plan['battery'][:t_hour+1])
            bat_action_required = target_soc_end - current_soc
            
            # Open-Loop execution for Heat Pump (Simpler)
            hp_action = self.plan['heat_pump'][t_hour]
            
            return {'battery': bat_action_required, 'heat_pump': hp_action}
        else:
            return {'battery': 0.0, 'heat_pump': 0.0}

    def _run_optimization(self, horizon=10000, initial_soc=0.0, initial_heat_soc=0.0):
        # 1. Get Full Forecast (Hourly)
        full_data = self.env.get_hourly_forecast(horizon=horizon)
        timesteps = len(full_data)
        
        if timesteps < 2:
            return {'battery': np.zeros(timesteps), 'heat_pump': np.zeros(timesteps)}

        # 2. Construct JSON Payload
        solar_profile = full_data['solar'].values
        max_solar = solar_profile.max()
        if max_solar == 0: max_solar = 1.0
        norm_solar = (solar_profile / max_solar).tolist()
        
        load_profile = full_data['load'].tolist()
        price_profile = full_data['price'].tolist()
        sell_price_profile = full_data['sell_price'].tolist()
        heat_demand_profile = full_data['heat_demand'].tolist()
        
        payload ={
            "site": {
                "Main": {
                    "area": 100,
                    "process": {
                        "Purchase": {
                            "wacc": 0, "cap-lo": 0, "cap-up": 1000, "fix-cost": 0, "inst-cap": 1000, "inv-cost": 0, "max-grad": "inf", "var-cost": 0,
                            "commodity": {"Elec": {"ratio": 1, "Direction": "Out", "ratio-min": 1}, "Elec buy": {"ratio": 1, "Direction": "In", "ratio-min": 1}},
                            "description": "Buy electricity", "depreciation": 50, "min-fraction": 0
                        },
                        "Feed-in": {
                            "wacc": 0, "cap-lo": 0, "cap-up": 1000, "fix-cost": 0, "inst-cap": 1000, "inv-cost": 0, "max-grad": "inf", "var-cost": 0,
                            "commodity": {"Elec": {"ratio": 1, "Direction": "In", "ratio-min": 1}, "Elec sell": {"ratio": 1, "Direction": "Out", "ratio-min": 1}},
                            "description": "Sell electricity", "depreciation": 50, "min-fraction": 0
                        },
                        "Photovoltaics": {
                            "wacc": 0.07, "cap-lo": max_solar, "cap-up": max_solar, "fix-cost": 0, "inst-cap": max_solar, "inv-cost": 0, "max-grad": "inf", "var-cost": 0,
                            "commodity": {"Elec": {"ratio": 1, "Direction": "Out", "ratio-min": 1}, "Solar": {"ratio": 1, "Direction": "In", "ratio-min": 1}},
                            "description": "PV", "area-per-cap": 5, "depreciation": 25, "min-fraction": 0
                        },
                        "HeatPump": {
                             "wacc": 0.07,
                             "cap-lo": 0,
                             "cap-up": self.env.hp_max_power * self.env.cop, # Capacity on Output (Heat)
                             "inst-cap": self.env.hp_max_power * self.env.cop,
                             "fix-cost": 0, "inv-cost": 0, "max-grad": "inf", "var-cost": 0,
                             "commodity": {
                                 "Elec": {"ratio": 1/self.env.cop, "Direction": "In"},
                                 "Heat": {"ratio": 1, "Direction": "Out"}
                             },
                             "description": "Heat Pump", "depreciation": 20, "min-fraction": 0
                        }
                    },
                    "commodity": {
                        "Elec": {
                            "Type": "Demand", "unitC": "kWh", "unitR": "kW", "demand": load_profile,
                            "storage": {
                                "Lead-Acid Battery": {
                                    "init": initial_soc / self.bat_cap if self.bat_cap > 0 else 0,
                                    "wacc": 0.007, "eff-in": 1, "eff-out": 1,
                                    "cap-lo-c": self.bat_cap, "cap-lo-p": self.max_p,
                                    "cap-up-c": self.bat_cap, "cap-up-p": self.max_p,
                                    "discharge": 0, "fix-cost-c": 0, "fix-cost-p": 0,
                                    "inst-cap-c": self.bat_cap, "inst-cap-p": self.max_p,
                                    "inv-cost-c": 0, "inv-cost-p": 0, "var-cost-c": 0, "var-cost-p": 0,
                                    "description": "Battery", "depreciation": 5
                                }
                            }
                        },
                        "Heat": {
                            "Type": "Demand", "unitC": "kWh", "unitR": "kW", "demand": heat_demand_profile,
                            "storage": {
                                "WaterTank": {
                                    "init": initial_heat_soc / self.env.heat_storage_capacity if self.env.heat_storage_capacity > 0 else 0,
                                    "wacc": 0.007, "eff-in": 1, "eff-out": 1,
                                    "cap-lo-c": self.env.heat_storage_capacity, "cap-lo-p": 1000,
                                    "cap-up-c": self.env.heat_storage_capacity, "cap-up-p": 1000,
                                    "discharge": 0, "fix-cost-c": 0, "fix-cost-p": 0,
                                    "inst-cap-c": self.env.heat_storage_capacity, "inst-cap-p": 1000,
                                    "inv-cost-c": 0, "inv-cost-p": 0, "var-cost-c": 0, "var-cost-p": 0,
                                    "description": "Heat Storage", "depreciation": 20
                                }
                            }
                        },
                        "Solar": {"Type": "SupIm", "supim": norm_solar, "unitC": "kWh", "unitR": "kW"},
                        "Elec buy": {"max": "inf", "Type": "Buy", "price": 0.1, "unitC": "kWh", "unitR": "kW", "maxperhour": "inf"},
                        "Elec sell": {"max": "inf", "Type": "Sell", "price": 0.0, "unitC": "kWh", "unitR": "kW", "maxperhour": "inf"}
                    }
                }
            },
            "global": {"CO2 limit": 150000000, "Cost limit": 35000000000},
            "c_timesteps": timesteps,
            "buysellprice": {"Elec buy": price_profile, "Elec sell": sell_price_profile}
        }
        
        # 3. Send Request
        with open('adg_payload.urbs', 'w') as f:
            json.dump(payload, f, indent=2)
        response = requests.post(self.url, json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        
        # 4. Parse Result
        # Battery
        storage_res = result['data']['results']['Main']['Elec']['storage']
        charge = np.array(storage_res['Stored'])
        discharge = np.array(storage_res['Retrieved'])
        bat_actions = charge - discharge
        
        # Heat Pump
        try:
            hp_heat_out = np.array(result['data']['results']['Main']['HeatPump']['commodity']['Heat']['Out'])
            hp_elec_in = hp_heat_out / self.env.cop
        except KeyError:
            hp_elec_in = np.zeros(timesteps)
            
        return {'battery': bat_actions, 'heat_pump': hp_elec_in}

In [ ]:
class LimitedURBSOptimizer(UrbsMPCOptimizer):
    """MPC optimizer using Urbs for optimization with perfect foresight and limited horizon."""
    def __init__(self, env, url="http://localhost:5000/simulate"):
        super().__init__(env, url)
        self.horizon = 1000

    def get_setpoint(self, observation, current_soc, current_heat_soc=None):
        if current_heat_soc is None: current_heat_soc = 0.0
        actions = self._run_optimization(horizon=self.horizon, initial_soc=current_soc, initial_heat_soc=current_heat_soc)
        
        # We only execute the first action of the plan
        if len(actions['battery']) > 0:
            return {'battery': actions['battery'][0], 'heat_pump': actions['heat_pump'][0]}
        else:
            return {'battery': 0.0, 'heat_pump': 0.0}

# ==========================================
# 3. REAL-TIME CONTROLLER (Minute-Level)
# ==========================================

In [ ]:
class RealTimeController:
    """
    Runs every minute to execute the hourly setpoint.
    """
    def __init__(self, env):
        self.env = env
    
    def get_action(self, setpoint, observation):
        """
        Determine the minute-level action.
        Args:
            setpoint: dict {'battery': kW, 'heat_pump': kW, 'pv_limit': kW} or float (legacy)
            observation (Series): Current minute-level observation (load, solar, etc.).
        Returns:
            dict: {'battery': kW, 'heat_pump': kW, 'pv_limit': kW}
        """
        if isinstance(setpoint, dict):
            # Ensure all keys are present
            return {
                'battery': setpoint.get('battery', 0.0),
                'heat_pump': setpoint.get('heat_pump', 0.0),
                'pv_limit': setpoint.get('pv_limit', float('inf'))
            }
        else:
            return {'battery': float(setpoint), 'heat_pump': 0.0, 'pv_limit': float('inf')}

In [ ]:
class InterpolatingRealTimeController(RealTimeController):
    """
    Interpolates between the previous setpoint and the current one over the course of the hour.
    """
    def __init__(self, env):
        super().__init__(env)
        self.last_setpoint = {'battery': 0.0, 'heat_pump': 0.0, 'pv_limit': float('inf')}
        self.target_setpoint = {'battery': 0.0, 'heat_pump': 0.0, 'pv_limit': float('inf')}
        self.steps_since_change = 0
        self.interpolation_steps = 60 # Interpolate over 1 hour

    def get_action(self, setpoint, observation):
        # Normalize setpoint to dict
        if not isinstance(setpoint, dict):
            setpoint = {'battery': float(setpoint), 'heat_pump': 0.0, 'pv_limit': float('inf')}
        
        if 'pv_limit' not in setpoint:
            setpoint['pv_limit'] = float('inf')
            
        # Check if setpoint has changed
        if setpoint != self.target_setpoint:
            # If it's the very first step, start at the setpoint to avoid ramping from 0
            if self.env.time_step == 0:
                self.last_setpoint = setpoint
            else:
                self.last_setpoint = self.target_setpoint
                
            self.target_setpoint = setpoint
            self.steps_since_change = 0
        
        # Calculate interpolation
        if self.steps_since_change < self.interpolation_steps:
            alpha = self.steps_since_change / self.interpolation_steps
            bat_action = self.last_setpoint['battery'] * (1 - alpha) + self.target_setpoint['battery'] * alpha
            hp_action = self.last_setpoint['heat_pump'] * (1 - alpha) + self.target_setpoint['heat_pump'] * alpha
            # For PV limit, we switch immediately to the target to ensure compliance
            pv_limit = self.target_setpoint['pv_limit']
        else:
            bat_action = self.target_setpoint['battery']
            hp_action = self.target_setpoint['heat_pump']
            pv_limit = self.target_setpoint['pv_limit']
            
        self.steps_since_change += 1
        return {'battery': bat_action, 'heat_pump': hp_action, 'pv_limit': pv_limit}

In [ ]:
class S2FillRateController(RealTimeController):
    """
    Implements S2 Standard Fill Rate Based Control (FRBC).
    Translates the hourly setpoint (kW) into an S2 'Operation Mode Factor',
    then converts that factor back to a fill rate (kW) to control the battery.
    Enforces local capacity constraints (mimicking device self-protection).
    """
    def __init__(self, env):
        super().__init__(env)
        # Define S2 Operation Mode limits (from Env)
        self.max_discharge_rate = -env.max_power # kW (Factor 0.0)
        self.max_charge_rate = env.max_power     # kW (Factor 1.0)
        
    def get_action(self, setpoint, observation):
        # Parse setpoint
        if isinstance(setpoint, dict):
            bat_setpoint_kw = setpoint.get('battery', 0.0)
            hp_setpoint_kw = setpoint.get('heat_pump', 0.0)
            pv_limit_kw = setpoint.get('pv_limit', float('inf'))
        else:
            bat_setpoint_kw = float(setpoint)
            hp_setpoint_kw = 0.0
            pv_limit_kw = float('inf')

        # --- 1. CEM Side: Determine Operation Mode Factor ---
        # Map requested kW to [0.0, 1.0]
        # Factor = (Request - Min) / (Max - Min)
        power_range = self.max_charge_rate - self.max_discharge_rate
        if power_range <= 0:
            factor = 0.5
        else:
            factor = (bat_setpoint_kw - self.max_discharge_rate) / power_range
            factor = np.clip(factor, 0.0, 1.0)
            
        # --- 2. Resource Manager Side: Execute Factor ---
        # Map Factor back to Fill Rate (kW)
        fill_rate_kw = self.max_discharge_rate + factor * power_range
        
        # --- 3. Local Constraints (S2 Safety) ---
        # Ensure we don't violate SoC limits locally before sending to physics
        # (This mimics the device protecting itself)
        current_soc = self.env.soc
        capacity = self.env.battery_capacity
        
        # Predicted energy change for this minute
        energy_delta = fill_rate_kw / 60.0
        predicted_soc = current_soc + energy_delta
        
        if predicted_soc > capacity:
            # Cap charge rate to just fill the battery
            max_energy_in = capacity - current_soc
            fill_rate_kw = max_energy_in * 60.0
        elif predicted_soc < 0:
            # Cap discharge rate to just empty the battery
            max_energy_out = -current_soc
            fill_rate_kw = max_energy_out * 60.0
            
        return {'battery': fill_rate_kw, 'heat_pump': hp_setpoint_kw, 'pv_limit': pv_limit_kw}

# ==========================================
# 4. Experiment
# ==========================================

In [ ]:
def run_experiment(variant_tuple):
    OptimizerClass, ControllerClass = variant_tuple
    # Setup
    env = SmartHomeEnv(hours=24)
    optimizer = OptimizerClass(env)
    rt_controller = ControllerClass(env)

    # History Storage
    results = []

    # Start
    obs = env.reset()
    done = False
    
    hourly_setpoint = {'battery': 0.0, 'heat_pump': 0.0}

    print(f"Starting Simulation with {OptimizerClass.__name__} + {ControllerClass.__name__}...")

    while not done:
        # 1. Hourly Optimization (Run once at the start of each hour)
        if env.time_step % 60 == 0:
            current_hour = env.time_step // 60
            # Get hourly observation for the UPCOMING hour
            if current_hour < len(env.data_hourly):
                obs_hourly = env.data_hourly.iloc[current_hour]
                # Optimizer decides setpoint for this hour
                hourly_setpoint = optimizer.get_setpoint(obs_hourly, env.soc, env.heat_soc)
            else:
                hourly_setpoint = {'battery': 0.0, 'heat_pump': 0.0}

        # 2. Real-Time Control (Every Minute)
        action_requested = rt_controller.get_action(hourly_setpoint, obs)

        # 3. Environment reacts
        next_obs, cost, done, info = env.step(action_requested)

        # 4. Store Data
        if isinstance(action_requested, dict):
            info['action_requested_bat'] = action_requested.get('battery', 0.0)
            info['action_requested_hp'] = action_requested.get('heat_pump', 0.0)
        else:
            info['action_requested_bat'] = action_requested
            info['action_requested_hp'] = 0.0
            
        if isinstance(hourly_setpoint, dict):
            info['setpoint_bat'] = hourly_setpoint.get('battery', 0.0)
            info['setpoint_hp'] = hourly_setpoint.get('heat_pump', 0.0)
        else:
            info['setpoint_bat'] = hourly_setpoint
            info['setpoint_hp'] = 0.0
            
        info['cost'] = cost
        # Calculate thermal power for visualization
        info['heat_power_thermal'] = info['hp_power'] * env.cop
        
        results.append(info)

        # 5. Advance
        obs = next_obs

    # Process Results
    df_res = pd.DataFrame(results)
    return df_res

# ==========================================
# 5. VISUALIZATION
# ==========================================

In [ ]:
def plot_controller_performance(df, controller_name, total_cost):
    print(f"Total Cost for {controller_name}: €{total_cost:.2f}")

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)

    # Ax1: Electric Power Flows
    ax1.set_title(f"{controller_name}: Electric Power Flows")
    ax1.plot(df['load'], 'k--', label='Load', alpha=0.5)
    ax1.plot(df['solar'], 'orange', label='Solar', alpha=0.5)
    ax1.plot(df['hp_power'], 'r', label='Heat Pump (Elec)', alpha=0.6)
    ax1.bar(df.index, df['battery_action_actual'], color='green', alpha=0.3, label='Battery Flow')
    ax1.legend()
    ax1.set_ylabel("kW")

    # Ax2: Battery State
    ax2.set_title(f"{controller_name}: Battery State of Charge")
    ax2.plot(df['soc'], 'g-', linewidth=2)
    ax2.set_ylabel("kWh")
    ax2.grid(True, alpha=0.3)
    
    # Ax3: Heat State
    ax3.set_title(f"{controller_name}: Heat Storage & Demand")
    ax3.plot(df['heat_soc'], 'r-', linewidth=2, label='Heat Storage')
    ax3.plot(df['heat_demand'], 'k:', label='Heat Demand', alpha=0.5)
    ax3.set_ylabel("kWh / kW")
    ax3.set_xlabel("Minute")
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# ==========================================
# 6. COMPARISON
# ==========================================

In [ ]:
variants = {
    "ResidualCharge": (ResidualChargeOptimizer, RealTimeController),
    "RuleBased": (RuleBasedOptimizer, RealTimeController),
    # "OneShotURBS": (UrbsMPCOptimizer, RealTimeController),
    # "LimitedURBS": (LimitedURBSOptimizer, RealTimeController),
    # "InterpolatedURBS": (UrbsMPCOptimizer, InterpolatingRealTimeController),
    # "S2_URBS": (UrbsMPCOptimizer, S2FillRateController),
}

all_results = {}
for name, variant in variants.items():
    all_results[name] = run_experiment(variant)


fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
# Use the first result to get the index (which is now minute-level)
df_env = list(all_results.values())[0]

# Ax1: Environment & Physics
ax1.set_title("Power Flows (Minute Resolution)")
ax1.plot(df_env['solar_potential'] if 'solar_potential' in df_env else df_env['solar'], color='gold', label='Solar Potential', alpha=0.8)
ax1.plot(df_env['load'], color='black', linestyle='-', label='Load', alpha=0.6)
ax1.plot(df_env['heat_demand'], color='darkred', linestyle='-', label='Heat Demand', alpha=0.6)
# ax1.plot(df_env['price'], label='Price', alpha=0.5)

# Define colors for variants to stay within "Blue" for Bat and "Red" for Heat
variant_colors = {
    'ResidualCharge': {'bat': 'navy', 'heat': 'darkred'},
    'RuleBased': {'bat': 'dodgerblue', 'heat': 'crimson'},
}
fallback_colors = [
    {'bat': 'blue', 'heat': 'red'},
    {'bat': 'cyan', 'heat': 'magenta'},
    {'bat': 'teal', 'heat': 'pink'}
]

# Create twin axes for SoC
ax2_soc = ax2.twinx()
ax3_soc = ax3.twinx()

for i, (name, df) in enumerate(all_results.items()):
    total_cost = df['cost'].sum()
    
    # Get colors
    if name in variant_colors:
        c_bat = variant_colors[name]['bat']
        c_heat = variant_colors[name]['heat']
    else:
        idx = i % len(fallback_colors)
        c_bat = fallback_colors[idx]['bat']
        c_heat = fallback_colors[idx]['heat']
    
    # Ax1: Curtailed Solar
    if 'curtailed_solar' in df.columns and df['curtailed_solar'].sum() > 0.1:
        ax1.plot(df['curtailed_solar'], linestyle=':', color='orange', linewidth=2, label=f'{name} Curtailed')

    # Ax2: Battery (Blue)
    # Power (Left Axis)
    ax2.plot(df.index, df['battery_action_actual'], linestyle='--', color=c_bat, alpha=0.5, label=f'{name} Power')
    # SOC (Right Axis)
    ax2_soc.plot(df['soc'], linestyle='-', color=c_bat, linewidth=2, label=f"{name} SOC ({total_cost:.2f}€)")

    # Ax3: Heat (Red)
    # Power (Left Axis) - Plotting Thermal Power now!
    ax3.plot(df['heat_power_thermal'], linestyle='--', color=c_heat, alpha=0.5, label=f"{name} Thermal Power")
    # SOC (Right Axis)
    ax3_soc.plot(df['heat_soc'], linestyle='-', color=c_heat, linewidth=2, label=f"{name} SOC")

ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.set_ylabel("kW")
ax1.grid(True, alpha=0.3)

ax2.set_title("Battery (Blue)")
ax2.set_ylabel("Power (kW)")
ax2_soc.set_ylabel("Energy (kWh)")
ax2.grid(True, alpha=0.3)
# Combine legends
lines2, labels2 = ax2.get_legend_handles_labels()
lines2s, labels2s = ax2_soc.get_legend_handles_labels()
ax2.legend(lines2 + lines2s, labels2 + labels2s, bbox_to_anchor=(1.05, 1), loc='upper left')

ax3.set_title("Heat (Red)")
ax3.set_ylabel("Thermal Power (kW)")
ax3_soc.set_ylabel("Energy (kWh)")
ax3.set_xlabel("Minute")
ax3.grid(True, alpha=0.3)
# Combine legends
lines3, labels3 = ax3.get_legend_handles_labels()
lines3s, labels3s = ax3_soc.get_legend_handles_labels()
ax3.legend(lines3 + lines3s, labels3 + labels3s, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()